In [1]:
import numpy as np
import copy

In [2]:
file_pth = './data.txt'

In [3]:
def read_file(path):
    data = []
    with open(path) as file:
        for row in file:
            data.append(np.array(row.split(),dtype="str"))
    return data

In [4]:
file = np.array(read_file(file_pth))
DATA = file[1:]
features = file[0][:-1]

In [5]:
np.unique(DATA[:,-1])

array(['buy', 'coffeeshop', 'home', 'mountaineer'], dtype='<U11')

In [6]:
DATA

array([['sunny', 'True', 'False', 'buy'],
       ['sunny', 'True', 'True', 'coffeeshop'],
       ['snowy', 'True', 'False', 'coffeeshop'],
       ['sunny', 'True', 'True', 'mountaineer'],
       ['rainy', 'True', 'True', 'home'],
       ['sunny', 'False', 'False', 'mountaineer'],
       ['sunny', 'True', 'False', 'mountaineer'],
       ['rainy', 'True', 'True', 'coffeeshop'],
       ['snowy', 'True', 'True', 'buy'],
       ['snowy', 'False', 'True', 'home']], dtype='<U11')

In [7]:
class DecisionTree():
    def __init__(self,DATA,features):
        self.data = copy.deepcopy(DATA)
        self.features = features
        self.rules = []
        self.tree = []
        
        self.calc_tree(self.data,[i for i in range(self.data.shape[1]-1)])
        self.print_tree()

    def calc_tree(self,data,columns):
        # check single result
        if(len(np.unique(data[:,-1])) <= 1):
           self.tree.append("$")
           self.tree.append(np.unique(data[:,-1])[0])
           return 0
        
        # check no feature
        if(columns == 0):
            vals , count = np.unique(data[:,-1] , return_counts=True)
            res = ""
            for i in range(len(vals)):
                res += vals[i]
                res += " : "
                res += str(count[i]/np.sum(count)*100)
                res += "%\t"
            self.tree.append("$")
            self.tree.append(res)
            return 0
        
        column = copy.deepcopy(columns)
        data_size = data.shape[0]
        best_entropy = self.Entropy(data)
        best_index = -1

        for i in column:
            a = data[data[:, i].argsort()]
            splited_data = np.split(a[:,:], np.unique(a[:,i], return_index=True)[1][0:])
            entropy = 0

            for x in splited_data:
                try:
                    entropy += (len(x) / data_size ) * self.Entropy(x)
                except:
                    pass
            if(entropy < best_entropy ):
                best_entropy = entropy
                best_index = i
            
        if(best_index != -1 ):
            a = data[data[:, best_index].argsort()]
            vals = np.unique(data[:,best_index])
            a = np.split(a[:,:], np.unique(a[:,best_index], return_index=True)[1][0:])[1:]
            column.remove(best_index)
            self.tree.append(">")
            self.tree.append(best_index)
            self.tree.append(":")

            for i in range(len(a)):
                self.tree.append(vals[i])

                self.calc_tree(a[i],column)
            self.tree.append("<")
        else:
            vals , count = np.unique(data[:,-1] , return_counts=True)
            res = ""
            for i in range(len(vals)):
                res += vals[i]
                res += " : "
                res += str(count[i]/np.sum(count)*100)
                res += "%\t"
            self.tree.append("$")
            self.tree.append(res)
    
    def Entropy(delf,data):
        _ , count = np.unique(data[:,-1],return_counts=True)
        count = count / np.sum(count)
        entropy = np.multiply(count,np.log2(count))
        total_entropy = -np.sum(entropy)
        return total_entropy
    
    def print_tree(self):
        i = 0
        flag = 0
        for x in self.tree:
            if(x == ">"):
                # i +=1
                flag = 1
            elif(x == "<"):
                i -= 1
                print("\t"*i,"}")
            elif(x == ":"):
                i += 1
            elif(x == "$"):
                i += 1
                flag = 2
            else:
                if(flag == 1):
                    print("\t"*i,"-" ,self.features[x] , " { ")
                elif(flag == 2):
                    print("\t"*i,x)
                    i -= 1
                else:
                    print("\t"*i ,x," : ")
                flag = 0

    
    def extract_rules(self,tree,rule):
        if(len(tree) == 0):
            return 1


        if(tree[0] == '>'):
            tree.remove('>')
            key = tree[0]
            tree.remove(':')
            val = tree[1]
            tree.remove(tree[0])
            tree.remove(tree[0])
            next_rule = copy.deepcopy(rule)
            next_rule.append(key)
            next_rule.append(val)
            self.extract_rules(copy.deepcopy(tree),next_rule)
        
        elif(tree[0] == '$'):
            tree.remove(tree[0])
            val = tree[0]
            tree.remove(tree[0])
            next_rule = copy.deepcopy(rule)
            next_rule.append(val)
            self.rules.append(next_rule)
            rule.pop()
            self.extract_rules(copy.deepcopy(tree),rule)
           

        elif(tree[0] == '<'):
            if(len(tree) > 1 and type(tree[1]) != int):
                rule.pop()
            rule.pop()
            tree.remove(tree[0])
            self.extract_rules(copy.deepcopy(tree),rule)

        else:
            val = tree[0]
            tree.remove(val)
            next_rule = copy.deepcopy(rule)
            next_rule.append(val)
            self.extract_rules(tree,next_rule)
           


    def print_rules(self):
        for rule in self.rules:
            rule_size = len(rule)
            i = 0
            while(i<rule_size-3):
                print(f"if ( {self.features[rule[i]]} == {rule[i+1]} ) and " ,end="" )
                i += 2
            print(f"if ( {self.features[rule[i]]} == {rule[i+1]} ) then {rule[i+2]}")
            

            


In [8]:
dt = DecisionTree(DATA,features)

 - weather  { 
	 rainy  : 
		 coffeeshop : 50.0%	home : 50.0%	
	 snowy  : 
	 - has_money  { 
		 False  : 
			 home
		 True  : 
		 - exam  { 
			 False  : 
				 coffeeshop
			 True  : 
				 buy
		 }
	 }
	 sunny  : 
	 - exam  { 
		 False  : 
		 - has_money  { 
			 False  : 
				 mountaineer
			 True  : 
				 buy : 50.0%	mountaineer : 50.0%	
		 }
		 True  : 
			 coffeeshop : 50.0%	mountaineer : 50.0%	
	 }
 }


In [9]:
dt.extract_rules(copy.deepcopy(dt.tree),[])
dt.print_rules()

if ( weather == rainy ) then coffeeshop : 50.0%	home : 50.0%	
if ( weather == snowy ) and if ( has_money == False ) then home
if ( weather == snowy ) and if ( has_money == True ) and if ( exam == False ) then coffeeshop
if ( weather == snowy ) and if ( has_money == True ) and if ( exam == True ) then buy
if ( weather == sunny ) and if ( exam == False ) and if ( has_money == False ) then mountaineer
if ( weather == sunny ) and if ( exam == False ) and if ( has_money == True ) then buy : 50.0%	mountaineer : 50.0%	
if ( weather == sunny ) and if ( exam == True ) then coffeeshop : 50.0%	mountaineer : 50.0%	
